Truth dataset HG002 version 3.2.2 or above is best to use for benchmarking.

how it was done
https://github.com/ga4gh/benchmarking-tools/tree/master/doc/ref-impl

In [ ]:
da = "/Users/kate/craft/data/"

re = joinpath(da, "grch/GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fna.gz")

vb = joinpath(da, "HG002_truth/HG002_GRCh38_1_22_v4.2.1_benchmark.vcf.gz")

vc = joinpath(da, "apply_germline_dna_to_genome/call_germline_variant/pass.vcf.gz")

sdf = joinpath(pai, "GRCh38.sdf")

In [ ]:
# Make vcfeval SDF
rtg = "/Users/kate/craft/workflow/Fastq.jl/tool/rtg-tools-3.11/rtg"

sd = joinpath(da, "grch/grch/GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.sdf")

run(`$rtg format -o $sdf $re`)

In [ ]:
# Configure vcfeval
run(`$rtg`)

In [ ]:
# ### Download Real Time Genomics prepard GRCH38 files
# vcfeval requires reference files in RTG formats. RTG makes versions of these files for the popular reference genomes such as GRCh38, which will be downloaded below.

if ispath(sdf)

    println("Skipping download becauSE RTG GRCh38 data directory already exists:\n $sdf")

else

    run(
        pipeline(
            `wget -q -P ../input/ https://s3.amazonaws.com/rtg-dataSEts/references/GRCh38.sdf.zip`,
            `unzip -d ../input/ ../input/GRCh38.sdf.zip`,
            `rm ../input/GRCh38.sdf.zip`,
        ),
    )
end

In [ ]:
# Run vcfeval
# This command compares the VCF you generated (calls) to the highly confident VCF generated by Genome in a Bottle (baseline).
run(`$rtg vcfeval
    --baSEline=$vb
    --calls=$vc
    --template=$sdf
    --output=$pab
    --threads=1
    `)

# See results
# Read more about how to interpret vcfeval results [here](https://cdn.rawgit.com/RealTimeGenomics/rtg-tools/master/installer/resources/tools/RTGOperationsManual/rtg_command_reference.html#vcfeval).

pasu = open(joinpath(pab, "summary.txt"))

su = read(pasu)

su